In [2]:
import qutip as qt
import scipy as sc
import pickle
import sys
import os
import numpy as np
import matplotlib as mpl
import JC

# Change some MPL settings
mpl.rcParams.update({'font.size':22})
mpl.use('Qt5Agg',warn=False, force=True)
import matplotlib.pyplot as plt

# Progress bar yay
from ipywidgets import FloatProgress

# Get library path for UCL repository
ucl_lib_path = "/home/dustyd/Documents/PostDoc/Repositories/MarsPythonAPI/ucl-measurement-code/LibModules"
sys.path.append(ucl_lib_path)

# Get UCL modules
import data_management as dm
import physical_constants as pc


### Implementing Mostafa's code in qutip

Here we check that the two codes work identically.

First the `get_order()` function, which determines the order of the energies from the bare energies of the system, described by the interaction free Hamiltonian:

$$\hat{H}_b = \sum E_k |k\rangle \langle k| + \hbar \omega_r \hat{a}^\dagger \hat{a}$$

where the qubit Hamiltonian is expressed in the energy eigenbasis.

In [5]:
import numpy as np
import math

def get_order(omega_r, qubit_energy_list):
    """Get the order of eigenenergies that diagonalizer produces.

    Use the bare energy of the system as reference.

    Args:
        omega_r (float): Resonator frequency
        qubit_energy_list (List[float]): Qubit energies

    Returns:
        (List[int]): Indices giving the proper order of the elements in
            qubit_energy_list.
    """
    tmax = len(qubit_energy_list)  # Maximum number of transmon levels
    order = np.zeros(tmax)

    # Bare energies of the system
    diag_bare = np.array([-i*omega_r + qubit_energy_list[i]
                          for i in range(tmax)])
    # Eigenenergies in the order produced by diagonalizer
    eigensolver_order = np.linalg.eigvalsh(np.diag(diag_bare))

    # Find where the diagonalizer puts the energies
    for i in range(tmax):
        index, = np.where(eigensolver_order==diag_bare[i])
        order[i] = index

    #Diagonalizer puts the ith energy level in order[i]th location
    return order.astype(int)

In [6]:
wr = 2.1
Eilist = np.array([0,3,4,9,10])
get_order(wr,Eilist)

array([0, 1, 2, 3, 4])

In [4]:
# Construct a dummy Hamiltonian with qutip that produces the above energies
Hb = np.sum([Eilist[i]*qt.basis(5,i)*qt.basis(5,i).dag() for i in range(len(Eilist))])-wr*qt.num(5)
Ei,psii = Hb.eigenstates()
print (Hb)
print (Ei)
print (psii)

Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = True
Qobj data =
[[ 0.   0.   0.   0.   0. ]
 [ 0.   0.9  0.   0.   0. ]
 [ 0.   0.  -0.2  0.   0. ]
 [ 0.   0.   0.   2.7  0. ]
 [ 0.   0.   0.   0.   1.6]]
[-0.2  0.   0.9  1.6  2.7]
[Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]
 [0.]]]


Since the bare qubit Hamitonian as defined above should always be diagonal, the order is actually encoded in the obtained basis states. We can use those to extract the order indices by constructing a number operator that follows the order of the obtained solutions:

$$\hat{n}' = \sum i|i\rangle \langle i|$$

and then projecting on the original number operator basis states:

$$\mathrm{index} = \langle k| \hat{n}' |k \rangle$$

This way no search is needed and it should handle degeneracy as well as the eigen solver does. Not sure if there is performance advantage with this. Possibly not when the matrices becomes large, will have to see.

In [5]:
op = np.sum([psii[i]*psii[i].dag()*i for i in range(5)])
order_vals = np.array([int((qt.basis(5,i).dag()*op*qt.basis(5,i))[0][0].real) for i in range(len(psii))])
print (order_vals)

[1 2 0 4 3]


In [6]:
def diagonalize_RWA_strip(tot_excit, omega_r, qubit_energy_list, g_list, order):
    """Diagonalize a single RWA strip

    Args:
        tot_excit (int): Total number of excitations in the strip
        omega_r (float): resonator frequency
        qubit_energy_list (List[float]): list of qubit energies.
        g_list (List[float]): list of qubit to resonator couplings. g_list[i] is
            qubit nearest neighbor between i+1 and i.
        order (List[int]): the order that the eigenstates should be sorted
    """
    # Maximum number of transmon levels, which also sets the size of the RWA
    # strip
    tmax = len(qubit_energy_list)
    # Diagonal elements of the RWA strip Hamiltonian
    diagonal_elements = np.array([(tot_excit-i)*omega_r + qubit_energy_list[i]
                                  for i in range(tmax)])
    # Off-diagonal elements of the RWA strip Hamiltonian
    offdiagonal_elements = np.array(
            [g_list[i]*math.sqrt((tot_excit-i)*(tot_excit-i>0))
             for i in range(tmax-1)])
    # Construct the total Hamiltonian of the RWA strip
    strip_H = (np.diag(diagonal_elements) + np.diag(offdiagonal_elements, 1) +
               np.diag(offdiagonal_elements, -1))
    
    # Get eigensystem
    eigenvalues, eigenvectors = np.linalg.eigh(strip_H)
    # Sort eigenvalues according to the order
    eigenvalues = np.array([eigenvalues[i] for i in order])
    # Sort eigenstates according to the order
    eigenvectors = np.array([eigenvectors[:,i] for i in order])

    # eigenvalues[q] corresponds to eigenenergy of the RWA strip with resonator
    # state 'tot_excit-q' and qubit state 'q'
    return [eigenvalues, eigenvectors]

In [7]:
n = 3
g_list = np.array([1.0,1.01,1.02,1.03])
order = get_order(wr,Eilist)
diagonalize_RWA_strip(n,wr,Eilist,g_list,order)

[array([5.99261035, 7.9       , 4.34065897, 9.52819192, 8.73853875]),
 array([[ 0.66074575, -0.11726354, -0.70210973,  0.23813074,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
        [ 0.53922431, -0.6099846 ,  0.56721871, -0.12417273,  0.        ],
        [ 0.22232794,  0.41437426,  0.40582294,  0.78369127,  0.        ],
        [-0.47246427, -0.66517819, -0.1435709 ,  0.56009185,  0.        ]])]

In Qutip we can define the coupling Hamiltonian as such:

$$\hat{H}_g = \sum^{N-1}_{i=0} g_i \sqrt{n - i} |i\rangle \langle i+1|$$

In [8]:
# Coupling Hamiltonian
Hg = np.sum([g_list[i]*np.sqrt((n-i)*(n-i>0))*qt.basis(5,i)*qt.basis(5,i+1).dag() for i in range(len(g_list))])
Hg += Hg.dag()

# Recompute bare Hamiltonian with offset in resonator occupation
Hbb = np.sum([Eilist[i]*qt.basis(5,i)*qt.basis(5,i).dag() for i in range(len(Eilist))])-wr*qt.num(5,-n)

# Final Hamiltonian
Hf = Hg + Hbb
Ein,psiin = Hf.eigenstates()
print (Hf)
print (Ein)
print (psiin)

Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = True
Qobj data =
[[6.3        1.73205081 0.         0.         0.        ]
 [1.73205081 7.2        1.4283557  0.         0.        ]
 [0.         1.4283557  6.1        1.02       0.        ]
 [0.         0.         1.02       9.         0.        ]
 [0.         0.         0.         0.         7.9       ]]
[4.34065897 5.99261035 7.9        8.73853875 9.52819192]
[Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[-0.53922431]
 [ 0.6099846 ]
 [-0.56721871]
 [ 0.12417273]
 [ 0.        ]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[-0.66074575]
 [ 0.11726354]
 [ 0.70210973]
 [-0.23813074]
 [ 0.        ]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[ 0.47246427]
 [ 0.66517819]
 [ 0.1435709 ]
 [-0.56009185]
 [ 0.        ]]
 Qu

In [9]:
# Now order the values
Eins = np.array([Ein[i] for i in order_vals])
psiins = np.array([psiin[i] for i in order_vals])
print (Eins)
print (psiins)

[5.99261035 7.9        4.34065897 9.52819192 8.73853875]
[Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[-0.66074575]
 [ 0.11726354]
 [ 0.70210973]
 [-0.23813074]
 [ 0.        ]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[-0.53922431]
 [ 0.6099846 ]
 [-0.56721871]
 [ 0.12417273]
 [ 0.        ]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.22232794]
 [0.41437426]
 [0.40582294]
 [0.78369127]
 [0.        ]]
 Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[ 0.47246427]
 [ 0.66517819]
 [ 0.1435709 ]
 [-0.56009185]
 [ 0.        ]]]


In [10]:
def diagonalize_ladder(nmax, omega_r, qubit_energy_list, g_list):
    """Diagonalize the JC ladder.

    Args:
        nmax (int): maximum number of photons in the resonator
        omega_r (float): resonator frequency.
        qubit_energy_list (List[float]): list of qubit energies
        g_list (List[float]): qubit charge matrix elements (qubit nearest
            neighbor couplings)
    """
    # Maximum number of transmon levels
    tmax = len(qubit_energy_list)
    # The order that should be used to sort the eigenenergies
    order = get_order(omega_r, qubit_energy_list)
    eigen_energies = np.zeros((nmax, tmax))
    eigen_vectors = np.zeros((nmax, tmax, tmax))
    # Get the egeinenergies of the whole ladder
    for i in range(nmax):
        eigen_energies[i], eigen_vectors[i] = diagonalize_RWA_strip(
                i, omega_r, qubit_energy_list, g_list, order)
    #First index of eigen_energies signifies the total excitation number of the
    # RWA strip and NOT the resonator state The conversion is: eigenenergy of
    # the system with resonator state 'n' and qubit state
    # 'q' <--> eigen_energies[n+q, q]
    return eigen_energies, eigen_vectors

# Now that we have the ordered eigenvalues and vectors for a strip we can compute the ladder in the same way
def qutip_JC(nmax,wr,Eilist,glist):
    
    # Get number of energy levels
    kmax = len(Eilist)
    
    # Get the ordering
    Hb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(len(Eilist))])-wr*qt.num(kmax)
    Ei,psii = Hb.eigenstates()
    op = np.sum([psii[i]*psii[i].dag()*i for i in range(kmax)])
    order = np.array([int((qt.basis(kmax,i).dag()*op*qt.basis(kmax,i))[0][0].real) for i in range(kmax)])
    
    # Construct full Hamiltonian for each n value
    eigva = []
    eigve = []
    for n in range(nmax):
        # Coupling Hamiltonian
        Hg = np.sum([glist[i]*np.sqrt((n-i)*(n-i>0))*qt.basis(kmax,i)*qt.basis(kmax,i+1).dag() for i in range(kmax-1)])
        Hg += Hg.dag()

        # Recompute bare Hamiltonian with offset in resonator occupation
        Hbb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(kmax)])-wr*qt.num(kmax,-n)

        # Final Hamiltonian
        Hf = Hg + Hbb
        e,v = Hf.eigenstates()
        
        # Order the values and save
        e = [e[i] for i in order]
        v = [v[i] for i in order]
        eigva.append(e)
        eigve.append(v)
    
    return np.array(eigva),eigve
    
    

In [11]:
diagonalize_ladder(2,wr,Eilist,g_list)

(array([[ 0.        ,  0.9       , -0.2       ,  2.7       ,  1.6       ],
        [ 1.9       ,  3.64658561,  1.45341439,  4.8       ,  3.7       ]]),
 array([[[ 1.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  1.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  1.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           1.        ]],
 
        [[ 0.        ,  0.        ,  1.        ,  0.        ,
           0.        ],
         [ 0.54297114,  0.83975136,  0.        ,  0.        ,
           0.        ],
         [-0.83975136,  0.54297114,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
         

In [12]:
qutip_JC(2,wr,Eilist,g_list)

(array([[ 0.        ,  0.9       , -0.2       ,  2.7       ,  1.6       ],
        [ 1.9       ,  3.64658561,  1.45341439,  4.8       ,  3.7       ]]),
 [[Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
   Qobj data =
   [[1.]
    [0.]
    [0.]
    [0.]
    [0.]], Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
   Qobj data =
   [[0.]
    [1.]
    [0.]
    [0.]
    [0.]], Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
   Qobj data =
   [[0.]
    [0.]
    [1.]
    [0.]
    [0.]], Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
   Qobj data =
   [[0.]
    [0.]
    [0.]
    [1.]
    [0.]], Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
   Qobj data =
   [[0.]
    [0.]
    [0.]
    [0.]
    [1.]]], [Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
   Qobj data =
   [[0.]
    [0.]
    [1.]
    [0.]
    [0.]], Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
   Qobj data =
   [[0.54297114]
    [

Now lets do some benchmarking to compare performance

In [13]:
# Generate large dummy data
nmax = 10
wr = 2.1
Eilist = [0,3]
g_list = [1.0]
acc = 3
n = 3
for i in range(120):
    if not i % 2:
        acc += 1
        Eilist.append(acc)
    else:
        acc += 5
        Eilist.append(acc)
    g_list.append(1.0+0.1*(i+1))

# functions to time
def test1():
    ret = diagonalize_ladder(nmax,wr,Eilist,g_list)
def test2():
    ret = qutip_JC(nmax,wr,Eilist,g_list)


import timeit
print(timeit.timeit("test1()", globals=globals(),number=1))
print(timeit.timeit("test2()", globals=globals(),number=1))

0.07534291000047233
2.0739003050002793


Hmm not very impressive, hopefully it's the ordering method used that forms a matrix that causes the problem

In [14]:
# Now that we have the ordered eigenvalues and vectors for a strip we can compute the ladder in the same way
def qutip_JC_mod(nmax,wr,Eilist,glist):
    
    # Get number of energy levels
    kmax = len(Eilist)
    
    # Get the ordering
    #Hb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(len(Eilist))])-wr*qt.num(kmax)
    #Ei,psii = Hb.eigenstates()
    #op = np.sum([psii[i]*psii[i].dag()*i for i in range(kmax)])
    order = get_order(wr,Eilist)
    
    # Construct full Hamiltonian for each n value
    eigva = []
    eigve = []
    for n in range(nmax):
        # Coupling Hamiltonian
        Hg = np.sum([glist[i]*np.sqrt((n-i)*(n-i>0))*qt.basis(kmax,i)*qt.basis(kmax,i+1).dag() for i in range(kmax-1)])
        Hg += Hg.dag()

        # Recompute bare Hamiltonian with offset in resonator occupation
        Hbb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(kmax)])-wr*qt.num(kmax,-n)

        # Final Hamiltonian
        Hf = Hg + Hbb
        e,v = Hf.eigenstates()
        
        # Order the values and save
        e = [e[i] for i in order]
        v = [v[i] for i in order]
        eigva.append(e)
        eigve.append(v)
    
    return np.array(eigva),eigve

In [15]:
def test3():
    ret = qutip_JC_mod(nmax,wr,Eilist,g_list)

print(timeit.timeit("test1()", globals=globals(),number=1))
print(timeit.timeit("test2()", globals=globals(),number=1))
print(timeit.timeit("test3()", globals=globals(),number=1))

0.006737527000950649
2.1496424789984303
1.9070392149988038


In [16]:
def test4():
    # Get number of energy levels
    kmax = len(Eilist)
    
    # Get the ordering
    Hb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(len(Eilist))])-wr*qt.num(kmax)
    Ei,psii = Hb.eigenstates()
    op = np.sum([psii[i]*psii[i].dag()*i for i in range(kmax)])
    order = np.array([int((qt.basis(kmax,i).dag()*op*qt.basis(kmax,i))[0][0].real) for i in range(kmax)])
def test5():
    ret = get_order(wr,Eilist)
print(timeit.timeit("test4()", globals=globals(),number=1))
print(timeit.timeit("test5()", globals=globals(),number=1))

0.2521864379996259
0.0005348439990484621


In [17]:
def test4():
    # Get the ordering
    kmax = len(Eilist)
    Hb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(len(Eilist))])-wr*qt.num(kmax)
    Ei,psii = Hb.eigenstates()
def test5():
    # Get the ordering
    kmax = len(Eilist)
    Hb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(len(Eilist))])-wr*qt.num(kmax)
    Ei = Hb.eigenenergies()
print(timeit.timeit("test4()", globals=globals(),number=1))
print(timeit.timeit("test5()", globals=globals(),number=1))

0.12686868099990534
0.07955827499972656


Ok so it looks like the generation of many `qt.basis` objects is likely the problem. How to fix?

In [18]:
qt.__file__

'/home/dustyd/anaconda3/lib/python3.6/site-packages/qutip/__init__.py'

Let's see how this kind of code performs in C, we can make a call whether to use qutip at all depending on the results.

First let's try `hello_world.pyx` from the [examples](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html):

In [19]:
import hello_world

Hello Jupyter!


Now we can see the annotated output for analysis:

In [20]:
%%HTML
<iframe width="100%" src="hello_world.html"></iframe>

Ok that worked now let's try a function that uses numpy. We should try to ensure that the cythonized file interacts as little as possible with python to ensure good speedups.

In [21]:
import numpy_test
numpy_test.solve(512)

array([-3.34817602e+01, -1.05264615e+01, -7.28636493e+00, -4.63154646e+00,
       -4.60445514e+00, -5.19624180e-14, -2.32113726e-14, -1.45913841e-14,
       -1.04675562e-14, -3.92223866e-15, -1.42097050e-15, -1.07096867e-15,
       -8.88178420e-16, -9.95574691e-17, -2.13223272e-18, -1.79503532e-18,
       -2.16659054e-19, -1.10733262e-19, -8.07465359e-21, -2.11311687e-24,
       -6.45209321e-30, -3.82039991e-30, -2.10328027e-30, -8.09232967e-31,
       -4.92238112e-31, -3.27559016e-31, -2.46805822e-31, -1.90179314e-31,
       -1.71263919e-31, -1.29652442e-31, -1.23942965e-31, -1.16501464e-31,
       -1.07048443e-31, -9.52417036e-32, -8.89669567e-32, -8.57710820e-32,
       -8.24460712e-32, -7.52154063e-32, -7.18908087e-32, -6.57467469e-32,
       -6.51306799e-32, -6.08411954e-32, -5.58107372e-32, -5.09256556e-32,
       -4.99211684e-32, -4.90873509e-32, -4.65515044e-32, -4.40188906e-32,
       -4.32871071e-32, -4.12139394e-32, -3.84716239e-32, -3.74643061e-32,
       -3.59519072e-32, -

In [22]:
%%HTML
<iframe width="100%" height="100%" src="numpy_test.html"></iframe>

As can be seen, every line interacts with python, so the speed benefit will not be impressive.

In [23]:
import numpy_test
numpy_test.solve(512)

array([-3.34817602e+01, -1.05264615e+01, -7.28636493e+00, -4.63154646e+00,
       -4.60445514e+00, -5.19624180e-14, -2.32113726e-14, -1.45913841e-14,
       -1.04675562e-14, -3.92223866e-15, -1.42097050e-15, -1.07096867e-15,
       -8.88178420e-16, -9.95574691e-17, -2.13223272e-18, -1.79503532e-18,
       -2.16659054e-19, -1.10733262e-19, -8.07465359e-21, -2.11311687e-24,
       -6.45209321e-30, -3.82039991e-30, -2.10328027e-30, -8.09232967e-31,
       -4.92238112e-31, -3.27559016e-31, -2.46805822e-31, -1.90179314e-31,
       -1.71263919e-31, -1.29652442e-31, -1.23942965e-31, -1.16501464e-31,
       -1.07048443e-31, -9.52417036e-32, -8.89669567e-32, -8.57710820e-32,
       -8.24460712e-32, -7.52154063e-32, -7.18908087e-32, -6.57467469e-32,
       -6.51306799e-32, -6.08411954e-32, -5.58107372e-32, -5.09256556e-32,
       -4.99211684e-32, -4.90873509e-32, -4.65515044e-32, -4.40188906e-32,
       -4.32871071e-32, -4.12139394e-32, -3.84716239e-32, -3.74643061e-32,
       -3.59519072e-32, -

In [24]:
%%HTML
<iframe width="100%" height="100%" src="numpy_test.html"></iframe>

Hmm although this is interesting, it will take a lot of work to reimplement numpy functions to minimize python interaction. Clearly the way to go if this is of interest is to use numerical C/C++ APIs such as GSL to handle these calculations.

In conclusion it is best to use the JC code as originally written, and integrating it into qutip by creating `Qobj` instances after the heavy lifting has been done. Lets have a go at this:

In [25]:
someH = np.array([[5.0,1.0],[1.0,-5.0]])
print (someH)
newH = qt.Qobj(inpt=someH,isherm=True)
print (newH)
print (newH.eigenstates())

[[ 5.  1.]
 [ 1. -5.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 5.  1.]
 [ 1. -5.]]
(array([-5.09901951,  5.09901951]), array([Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[ 0.09853762]
 [-0.99513333]],
       Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[-0.99513333]
 [-0.09853762]]], dtype=object))


In [26]:
def diagonalize_ladder_mod(nmax, omega_r, qubit_energy_list, g_list):
    
    # The order that should be used to sort the eigenenergies
    order = get_order(omega_r, qubit_energy_list)
    
    # Maximum number of transmon levels, which also sets the size of the RWA
    # strip
    tmax = len(qubit_energy_list)
    
    # Initialise
    eigen_energies = [0.0]*nmax
    eigen_vectors = [0.0]*nmax
    diagonal_elements = None
    offdiagonal_elements = None
    strip_H = None
    e = None
    v = None
    
    # Get the eigenenergies of the whole ladder
    for n in range(nmax):
        
        # Diagonal elements of the RWA strip Hamiltonian
        diagonal_elements = np.array([(n-i)*omega_r + qubit_energy_list[i]
                                      for i in range(tmax)])
        # Off-diagonal elements of the RWA strip Hamiltonian
        offdiagonal_elements = np.array(
                [g_list[i]*np.sqrt((n-i)*(n-i>0))
                 for i in range(tmax-1)])
        # Construct the total Hamiltonian of the RWA strip
        strip_H = qt.Qobj(np.diag(diagonal_elements) + np.diag(offdiagonal_elements, 1) +
                   np.diag(offdiagonal_elements, -1),isherm=True)

        # Get eigensystem
        e, v = strip_H.eigenstates()
        
        # Sort eigenvalues and vectors
        e = np.array([e[i] for i in order])
        v = np.array([v[i] for i in order])
        
        eigen_energies[n] = e
        eigen_vectors[n] = v
    
    return eigen_energies, eigen_vectors

In [27]:
# Generate large dummy data
nmax = 10
wr = 2.1
Eilist = [0,3]
g_list = [1.0]
acc = 3
n = 3
for i in range(120):
    if not i % 2:
        acc += 1
        Eilist.append(acc)
    else:
        acc += 5
        Eilist.append(acc)
    g_list.append(1.0+0.1*(i+1))

# functions to time
def test1():
    ret = diagonalize_ladder(nmax,wr,Eilist,g_list)
def test2():
    ret = diagonalize_ladder_mod(nmax,wr,Eilist,g_list)

import timeit
print(timeit.timeit("test1()", globals=globals(),number=1))
print(timeit.timeit("test2()", globals=globals(),number=1))

0.022020159998646704
0.5901722269991296


What causes this performance degradation here? Lets test the eigensolver:

In [28]:
a = np.zeros((512,512))
a[501] = a[32] = a[332] = a[154] = a[451] = 1
aq = qt.Qobj(a,isherm=True)
# functions to time
def test1():
    ret = np.linalg.eigh(a)
def test2():
    ret = aq.eigenstates()

import timeit
print(timeit.timeit("test1()", globals=globals(),number=100)/100.0)
print(timeit.timeit("test2()", globals=globals(),number=100)/100.0)

0.01393774972002575
0.2582763121800235


Ok, why is qutip version of diagonaliser so slow? It does use the scipy version of linalg, so lets check that:

In [29]:
a = np.zeros((512,512))
a[501] = a[32] = a[332] = a[154] = a[451] = 1
aq = qt.Qobj(a,isherm=True)
# functions to time
def test1():
    ret = np.linalg.eigh(a)
def test2():
    ret = sc.linalg.eig(a)
def test3():
    ret = sc.linalg.eigh(a)

import timeit
print(timeit.timeit("test1()", globals=globals(),number=100)/100.0)
print(timeit.timeit("test2()", globals=globals(),number=100)/100.0)
print(timeit.timeit("test3()", globals=globals(),number=100)/100.0)

0.013164845639985288
0.018324327299997095
0.010648965990003489


Ok so scipy version is actually a little faster on average, so it's not that either. So lets reproduce what qutip does a step at a time:

In [30]:
a = np.zeros((512,512))
a[501] = a[32] = a[332] = a[154] = a[451] = 1
aq = qt.Qobj(a,isherm=True)
# functions to time
def test1():
    ret = np.linalg.eigh(a)
def test2():
    evals,evecs = np.linalg.eigh(aq.data.todense())
    _zipped = list(zip(evals, range(len(evals))))
    _zipped.sort()
    evals, perm = list(zip(*_zipped))
    evecs = np.array([evecs[:, k] for k in perm])

import timeit
print(timeit.timeit("test1()", globals=globals(),number=100)/100.0)
print(timeit.timeit("test2()", globals=globals(),number=100)/100.0)

0.013109427060007874
0.03821989226002188


The performance hit for doing extra processing on the full set of eigenvalues and vectors is actually not that bad, I now suspect it has to do with the overhead of python interpreting multiple nested function and ifelse statements.

In [31]:
qt.settings.num_cpus

4

In [32]:
# Generate large dummy data
nmax = 10
wr = 2.1
Eilist = [0,3]
g_list = [1.0]
acc = 3
n = 3
for i in range(120):
    if not i % 2:
        acc += 1
        Eilist.append(acc)
    else:
        acc += 5
        Eilist.append(acc)
    g_list.append(1.0+0.1*(i+1))

# functions to time
def test1():
    ret = diagonalize_ladder(nmax,wr,Eilist,g_list)
def test2():
    ret = diagonalize_ladder_mod(nmax,wr,Eilist,g_list)

import timeit
print(timeit.timeit("test1()", globals=globals(),number=100)/100.0)
print(timeit.timeit("test2()", globals=globals(),number=100)/100.0)

0.005975871649970941
0.4882323188100054


In [33]:
def diagonalize_ladder(nmax, omega_r, qubit_energy_list, g_list):
    
    # The order that should be used to sort the eigenenergies
    order = get_order(omega_r, qubit_energy_list)
    
    # Maximum number of transmon levels, which also sets the size of the RWA
    # strip
    tmax = len(qubit_energy_list)
    
    # Initialise
    eigen_energies = [0.0]*nmax
    eigen_vectors = [0.0]*nmax
    diagonal_elements = None
    offdiagonal_elements = None
    strip_H = None
    e = None
    v = None
    
    # Get the eigenenergies of the whole ladder
    for n in range(nmax):
        
        # Diagonal elements of the RWA strip Hamiltonian
        diagonal_elements = np.array([(n-i)*omega_r + qubit_energy_list[i]
                                      for i in range(tmax)])
        # Off-diagonal elements of the RWA strip Hamiltonian
        offdiagonal_elements = np.array(
                [g_list[i]*np.sqrt((n-i)*(n-i>0))
                 for i in range(tmax-1)])
        # Construct the total Hamiltonian of the RWA strip
        strip_H = (np.diag(diagonal_elements) + np.diag(offdiagonal_elements, 1) +
                   np.diag(offdiagonal_elements, -1))

        # Get eigensystem
        e, v = np.linalg.eigh(strip_H)
        
        # Sort eigenvalues and vectors
        e = np.array([e[i] for i in order])
        v = np.array([v[i] for i in order])
        
        eigen_energies[n] = e
        eigen_vectors[n] = v
    
    return eigen_energies, eigen_vectors

def diagonalize_ladder_mod(nmax, omega_r, qubit_energy_list, g_list):
    
    # The order that should be used to sort the eigenenergies
    order = get_order(omega_r, qubit_energy_list)
    
    # Maximum number of transmon levels, which also sets the size of the RWA
    # strip
    tmax = len(qubit_energy_list)
    
    # Initialise
    eigen_energies = [0.0]*nmax
    eigen_vectors = [0.0]*nmax
    diagonal_elements = None
    offdiagonal_elements = None
    strip_H = None
    e = None
    v = None
    
    # Get the eigenenergies of the whole ladder
    for n in range(nmax):
        
        # Diagonal elements of the RWA strip Hamiltonian
        diagonal_elements = np.array([(n-i)*omega_r + qubit_energy_list[i]
                                      for i in range(tmax)])
        # Off-diagonal elements of the RWA strip Hamiltonian
        offdiagonal_elements = np.array(
                [g_list[i]*np.sqrt((n-i)*(n-i>0))
                 for i in range(tmax-1)])
        # Construct the total Hamiltonian of the RWA strip
        strip_H = qt.Qobj(np.diag(diagonal_elements) + np.diag(offdiagonal_elements, 1) +
                   np.diag(offdiagonal_elements, -1),isherm=True)

        # Get eigensystem
        #e, v = strip_H.eigenstates()
        # Get eigensystem
        e, v = sc.linalg.eigh(strip_H.data.todense())
        
        # Sort eigenvalues and vectors
        e = np.array([e[i] for i in order])
        v = np.array([v[i] for i in order])
        
        eigen_energies[n] = e
        eigen_vectors[n] = v
    
    return eigen_energies, eigen_vectors

In [34]:
# Generate large dummy data
nmax = 10
wr = 2.1
Eilist = [0,3]
g_list = [1.0]
acc = 3
n = 3
for i in range(120):
    if not i % 2:
        acc += 1
        Eilist.append(acc)
    else:
        acc += 5
        Eilist.append(acc)
    g_list.append(1.0+0.1*(i+1))

# functions to time
def test1():
    ret = diagonalize_ladder(nmax,wr,Eilist,g_list)
def test2():
    ret = diagonalize_ladder_mod(nmax,wr,Eilist,g_list)

import timeit
print(timeit.timeit("test1()", globals=globals(),number=100)/100.0)
print(timeit.timeit("test2()", globals=globals(),number=100)/100.0)

0.00723789329000283
0.0155779489699853


In [35]:
a = np.zeros((512,512))
a[501] = a[32] = a[332] = a[154] = a[451] = 1
aq = qt.Qobj(a,isherm=True)
# functions to time
def test1():
    a = np.zeros((512,512))
    a[501] = a[32] = a[332] = a[154] = a[451] = 1
def test2():
    a = np.zeros((512,512))
    a[501] = a[32] = a[332] = a[154] = a[451] = 1
    aq = qt.Qobj(a,isherm=True)

import timeit
print(timeit.timeit("test1()", globals=globals(),number=100)/100.0)
print(timeit.timeit("test2()", globals=globals(),number=100)/100.0)

9.719221998238936e-05
0.002705517720023636


Ok so let's avoid using `Qobj.eigenstates()` on Hamiltonians, the extra processing seems unnecessary

In [36]:
# Now that we have the ordered eigenvalues and vectors for a strip we can compute the ladder in the same way
def qutip_JC_mod(nmax,wr,Eilist,glist):
    
    # Get number of energy levels
    kmax = len(Eilist)
    
    # Get the ordering
    #Hb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(len(Eilist))])-wr*qt.num(kmax)
    #Ei,psii = Hb.eigenstates()
    #op = np.sum([psii[i]*psii[i].dag()*i for i in range(kmax)])
    order = get_order(wr,Eilist)
    
    # Construct full Hamiltonian for each n value
    eigva = []
    eigve = []
    for n in range(nmax):
        # Coupling Hamiltonian
        Hg = np.sum([glist[i]*np.sqrt((n-i)*(n-i>0))*qt.basis(kmax,i)*qt.basis(kmax,i+1).dag() for i in range(kmax-1)])
        Hg += Hg.dag()

        # Recompute bare Hamiltonian with offset in resonator occupation
        Hbb = np.sum([Eilist[i]*qt.basis(kmax,i)*qt.basis(kmax,i).dag() for i in range(kmax)])-wr*qt.num(kmax,-n)

        # Final Hamiltonian
        Hf = Hg + Hbb
        #e,v = Hf.eigenstates()
        e, v = sc.linalg.eigh(Hf.data.todense())
        
        # Order the values and save
        e = [e[i] for i in order]
        v = [v[i] for i in order]
        eigva.append(e)
        eigve.append(v)
    
    return np.array(eigva),eigve

In [37]:
# Generate large dummy data
nmax = 10
wr = 2.1
Eilist = [0,3]
g_list = [1.0]
acc = 3
n = 3
for i in range(120):
    if not i % 2:
        acc += 1
        Eilist.append(acc)
    else:
        acc += 5
        Eilist.append(acc)
    g_list.append(1.0+0.1*(i+1))

# functions to time
def test1():
    ret = diagonalize_ladder(nmax,wr,Eilist,g_list)
def test2():
    ret = qutip_JC_mod(nmax,wr,Eilist,g_list)

import timeit
print(timeit.timeit("test1()", globals=globals(),number=100)/100.0)
print(timeit.timeit("test2()", globals=globals(),number=100)/100.0)

0.007766123649998917
1.4085625700899982


Ok so the conclusion is clear: qutip shouldn't necessarily be used for repeatidly building Hamiltonians in it's current state. This is surely something of interest to doing full device simulations within qutip.

___

The final version of the code slightly optimized for fitting:

In [60]:
#
#  Jaynes-Cumming RWA strip diagonaliser, adapted from Mostafa's code
#
def diagonalize_ladder(nmax, omega_r, qubit_energy_list, g_list):
    
    # Maximum number of transmon levels, which also sets the size of the RWA
    # strip
    tmax = len(qubit_energy_list)
    
    # The order that should be used to sort the eigenenergies
    order = np.zeros(tmax,dtype=np.int)

    # Bare energies of the system
    diag_bare = np.array([-i*omega_r + qubit_energy_list[i]
                          for i in range(tmax)])
    # Eigenenergies in the order produced by diagonalizer
    eigensolver_order = np.linalg.eigvalsh(np.diag(diag_bare))

    # Find where the diagonalizer puts the energies
    for i in range(tmax):
        index, = np.where(eigensolver_order==diag_bare[i])
        order[i] = index
    
    # Initialise
    eigen_energies = [0.0]*nmax
    diagonal_elements = None
    offdiagonal_elements = None
    strip_H = None
    e = None
    
    # Get the eigenenergies of the whole ladder
    for n in range(nmax):
        
        # Diagonal elements of the RWA strip Hamiltonian
        diagonal_elements = np.array([(n-i)*omega_r + qubit_energy_list[i] for i in range(tmax)])
        
        # Off-diagonal elements of the RWA strip Hamiltonian
        offdiagonal_elements = np.array([g_list[i]*np.sqrt((n-i)*(n-i>0)) for i in range(tmax-1)])
        
        # Construct the total Hamiltonian of the RWA strip
        strip_H = (np.diag(diagonal_elements) + np.diag(offdiagonal_elements, 1) +
                   np.diag(offdiagonal_elements, -1))

        # Get eigensystem
        e = sc.linalg.eigvals(strip_H)
        
        # Sort eigenvalues and vectors
        eigen_energies[n] = np.array([e[i].real for i in order])
    
    return eigen_energies

In [61]:
# Generate large dummy data
nmax = 10
wr = 2.1
Eilist = [0,3]
g_list = [1.0]
acc = 3
n = 3
for i in range(120):
    if not i % 2:
        acc += 1
        Eilist.append(acc)
    else:
        acc += 5
        Eilist.append(acc)
    g_list.append(1.0+0.1*(i+1))

# functions to time
def test1():
    ret = diagonalize_ladder(nmax,wr,Eilist,g_list)
print(timeit.timeit("test1()", globals=globals(),number=100)/100.0)

0.005737937939993571


In [62]:
ret = diagonalize_ladder(nmax,wr,Eilist,g_list)

In [66]:
(ret[1][0] - ret[0][0])*1e9

2746585609.9730654

In [10]:
Eilist = np.array([0,3,4,9,10])
H1 = np.sum([Eilist[i]*qt.basis(5,i)*qt.basis(5,i).dag() for i in range(len(Eilist))])

In [31]:
H2 = 2.1*qt.num(5)

In [32]:
e1,v1 = H1.eigenstates()
e2,v2 = H2.eigenstates()

In [33]:
e2

array([0. , 2.1, 4.2, 6.3, 8.4])

In [34]:
qt.tensor(H1,H2).eigenstates()

(array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  6.3,  8.4,
        12.6, 16.8, 18.9, 18.9, 21. , 25.2, 25.2, 33.6, 37.8, 42. , 56.7,
        63. , 75.6, 84. ]),
 array([Quantum object: dims = [[5, 5], [1, 1]], shape = (25, 1), type = ket
 Qobj data =
 [[1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]],
        Quantum object: dims = [[5, 5], [1, 1]], shape = (25, 1), type = ket
 Qobj data =
 [[0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]],
        Quantum object: dims = [[5, 5], [1, 1]], shape = (25, 1), type = ket
 Qobj data =
 [[0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]],
        Quantum object: